In [3]:
import pickle
import gzip
import json
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from PIL import Image
import io
from projectaria_tools.core import data_provider, calibration

# =============================================================================
# Configuration
# =============================================================================
VRS_PATH = "data/b6591925-cd53-4ba9-b26c-eed4faffd8bd.vrs"
MPS_FOLDER = "data/point_clouds"
OUTPUT_PKL = "data/output.pkl"

# Object bounding box (you'll need to adjust these based on your object)
# Format: [x_min, y_min, z_min, x_max, y_max, z_max]
# For now, we'll compute from all points, but you should filter to your object
OBJECT_BBOX = None  # Set to None to use all points

# Number of views to sample
NUM_VIEWS = 16

# =============================================================================
# Helper Functions
# =============================================================================
def jpg_encode(image: np.ndarray) -> bytes:
    """Encode numpy image to JPEG bytes."""
    if len(image.shape) == 2:
        image = np.stack([image, image, image], axis=-1)
    image_pil = Image.fromarray(image.astype(np.uint8))
    buffer = io.BytesIO()
    image_pil.save(buffer, format="JPEG", quality=95)
    return buffer.getvalue()

def create_visible_points(points, images, camera_poses, intrinsics_list):
    """Create visibility map for points in each view."""
    visible_points = []
    
    for i, (img, pose, cam_params) in enumerate(zip(images, camera_poses, intrinsics_list)):
        H, W = img.shape[:2]
        
        # Build 3x3 intrinsic matrix from camera params
        # cam_params = [fx, fy, cx, cy, distortion...]
        K = np.array([
            [cam_params[0], 0, cam_params[2]],
            [0, cam_params[1], cam_params[3]],
            [0, 0, 1]
        ])
        
        # Transform points to camera space
        T_cam_world = np.linalg.inv(pose)
        points_cam = (T_cam_world[:3, :3] @ points.T).T + T_cam_world[:3, 3]
        
        # Project to image
        points_img = (K @ points_cam.T).T
        points_img = points_img / points_img[:, 2:3]
        
        # Check which points are visible
        visible = (
            (points_cam[:, 2] > 0) &  # In front of camera
            (points_img[:, 0] >= 0) & (points_img[:, 0] < W) &
            (points_img[:, 1] >= 0) & (points_img[:, 1] < H)
        )
        
        visible_indices = np.where(visible)[0]
        visible_points.append(visible_indices)
    
    return visible_points

def compute_point_projections(points, images, camera_poses, intrinsics_list, visible_points):
    """Compute 2D projections of 3D points for each view."""
    projections = []
    
    for i, (img, pose, cam_params, vis_idx) in enumerate(zip(images, camera_poses, intrinsics_list, visible_points)):
        H, W = img.shape[:2]
        
        if len(vis_idx) == 0:
            # No visible points, create empty projection
            projections.append(np.zeros((0, 2), dtype=np.float32))
            continue
        
        # Build 3x3 intrinsic matrix from camera params
        K = np.array([
            [cam_params[0], 0, cam_params[2]],
            [0, cam_params[1], cam_params[3]],
            [0, 0, 1]
        ])
        
        # Get visible points
        visible_pts = points[vis_idx]
        
        # Transform to camera space
        T_cam_world = np.linalg.inv(pose)
        points_cam = (T_cam_world[:3, :3] @ visible_pts.T).T + T_cam_world[:3, 3]
        
        # Project to image
        points_img = (K @ points_cam.T).T
        points_img = points_img[:, :2] / points_img[:, 2:3]
        
        projections.append(points_img.astype(np.float32))
    
    return projections

def load_mps_points(mps_folder):
    """Load semidense point cloud from MPS."""
    with gzip.open(f"{mps_folder}/semidense_points.csv.gz", 'rt') as f:
        points_df = pd.read_csv(f)
    
    points = points_df[['px_world', 'py_world', 'pz_world']].values
    
    # Load uncertainty if available
    if 'inv_dist_std' in points_df.columns:
        inv_dist_std = points_df['inv_dist_std'].values
        dist_std = points_df['dist_std'].values
    else:
        inv_dist_std = np.zeros(len(points))
        dist_std = np.zeros(len(points))
    
    return points, inv_dist_std, dist_std

def load_trajectory(mps_folder):
    """Load camera trajectory from MPS."""
    # Try closed loop first, fall back to open loop
    if Path(f"{mps_folder}/closed_loop_trajectory.csv").exists():
        traj_df = pd.read_csv(f"{mps_folder}/closed_loop_trajectory.csv")
    else:
        traj_df = pd.read_csv(f"{mps_folder}/open_loop_trajectory.csv")
    
    return traj_df

# Replace the extract_frames_and_poses function with this corrected version:
def extract_frames_and_poses(vrs_path, trajectory_df, num_views=16):
    """Extract frames and camera poses from VRS."""
    provider = data_provider.create_vrs_data_provider(vrs_path)
    
    # Get RGB camera stream
    rgb_stream_id = provider.get_stream_id_from_label("camera-rgb")
    
    # Sample frames uniformly
    total_frames = provider.get_num_data(rgb_stream_id)
    frame_indices = np.linspace(0, total_frames - 1, num_views, dtype=int)
    
    images = []
    camera_poses = []
    intrinsics_list = []
    timestamps = []
    
    # Get camera calibration
    device_calib = provider.get_device_calibration()
    rgb_calib = device_calib.get_camera_calib("camera-rgb")
    
    for idx in frame_indices:
        # Get image
        image_data = provider.get_image_data_by_index(rgb_stream_id, idx)
        image = image_data[0].to_numpy_array()
        
        # Convert to grayscale (ShapeR expects grayscale)
        if len(image.shape) == 3:
            image = np.mean(image, axis=2).astype(np.uint8)
        
        images.append(image)
        
        # Get timestamp
        timestamp_ns = image_data[1].capture_timestamp_ns
        timestamps.append(timestamp_ns)
        
        # Get intrinsics - ShapeR expects [fx, fy, cx, cy, k1, k2, k3, k4, ...]
        # For fisheye model with up to 16 parameters
        focal_lengths = rgb_calib.get_focal_lengths()
        principal_point = rgb_calib.get_principal_point()
        
        # Create camera params in expected format
        # Start with pinhole params [fx, fy, cx, cy]
        cam_params = np.array([
            focal_lengths[0],  # fx
            focal_lengths[1],  # fy
            principal_point[0],  # cx
            principal_point[1],  # cy
        ])
        
        # Add distortion coefficients if available (pad to 16 total)
        # Aria uses Kannala-Brandt fisheye model
        # For now, pad with zeros - you may need actual distortion coeffs
        distortion = np.zeros(12)  # Pad to make 16 total params
        cam_params = np.concatenate([cam_params, distortion])
        
        intrinsics_list.append(cam_params)
    
    # Match poses from trajectory
    for ts in timestamps:
        # Find closest pose in trajectory
        time_diff = np.abs(trajectory_df['tracking_timestamp_us'].values * 1000 - ts)
        closest_idx = np.argmin(time_diff)
        
        # Extract pose (rotation + translation)
        row = trajectory_df.iloc[closest_idx]
        
        # Camera to world transform
        T_world_cam = np.eye(4)
        T_world_cam[0, 3] = row['tx_world_device']
        T_world_cam[1, 3] = row['ty_world_device']
        T_world_cam[2, 3] = row['tz_world_device']
        
        # Rotation (quaternion to matrix)
        qw, qx, qy, qz = row['qw_world_device'], row['qx_world_device'], row['qy_world_device'], row['qz_world_device']
        R = quaternion_to_rotation_matrix(qw, qx, qy, qz)
        T_world_cam[:3, :3] = R
        
        camera_poses.append(T_world_cam)
    
    return images, camera_poses, intrinsics_list
    
def quaternion_to_rotation_matrix(qw, qx, qy, qz):
    """Convert quaternion to rotation matrix."""
    R = np.array([
        [1 - 2*(qy**2 + qz**2), 2*(qx*qy - qw*qz), 2*(qx*qz + qw*qy)],
        [2*(qx*qy + qw*qz), 1 - 2*(qx**2 + qz**2), 2*(qy*qz - qw*qx)],
        [2*(qx*qz - qw*qy), 2*(qy*qz + qw*qx), 1 - 2*(qx**2 + qy**2)]
    ])
    return R

def create_dummy_masks(images):
    """Create dummy masks (all ones - no segmentation)."""
    # You'll replace this with actual SAM2 masks
    masks = []
    for img in images:
        mask = np.ones_like(img) * 255
        masks.append(mask)
    return masks

# =============================================================================
# Main Conversion
# =============================================================================
def main():
    print("Loading MPS point cloud...")
    points, inv_dist_std, dist_std = load_mps_points(MPS_FOLDER)
    
    print(f"Loaded {len(points)} points")
    
    print("Loading trajectory...")
    trajectory_df = load_trajectory(MPS_FOLDER)
    
    print("Extracting frames and poses from VRS...")
    images, camera_poses, intrinsics_list = extract_frames_and_poses(
        VRS_PATH, trajectory_df, NUM_VIEWS
    )
    
    print(f"Extracted {len(images)} views")
    
    # Create masks (dummy for now - replace with SAM2)
    print("Creating masks (using dummy masks - replace with SAM2)...")
    masks = create_dummy_masks(images)
    
    # Compute point visibility
    print("Computing point visibility...")
    visible_points = create_visible_points(points, images, camera_poses, intrinsics_list)
    
    # Compute point projections
    print("Computing point projections...")
    point_projections = compute_point_projections(points, images, camera_poses, intrinsics_list, visible_points)
    
    # Compute bounds
    bounds = (points.max(axis=0) - points.min(axis=0)) / 2
    
    # Compute camera-to-model transforms (inverse of camera_to_worlds)
    Ts_camera_model = [np.linalg.inv(pose) for pose in camera_poses]
    
    # Build pickle
    print("Building pickle file...")
    pkl_sample = {
        # Point cloud
        "points_model": torch.from_numpy(points).float(),
        "bounds": torch.from_numpy(bounds).float(),
        
        # Uncertainty
        "inv_dist_std": torch.from_numpy(inv_dist_std).float(),
        "dist_std": torch.from_numpy(dist_std).float(),
        
        # Images and camera data
        "image_data": [jpg_encode(img) for img in images],
        "camera_to_worlds": [torch.from_numpy(pose).float() for pose in camera_poses],
        "Ts_camera_model": [torch.from_numpy(T).float() for T in Ts_camera_model],
        #"camera_params": [torch.from_numpy(K).float() for K in intrinsics_list],
        "camera_params": [torch.from_numpy(K).float() for K in intrinsics_list],
        "mask_data": [jpg_encode(mask) for mask in masks],
        
        # Visibility and projections
        "visible_points_model": visible_points,
        "object_point_projections": [torch.from_numpy(proj).float() for proj in point_projections],  # ADD THIS
        
        # Metadata
        "caption": "object from aria glasses",
    }
    
    # Save
    print(f"Saving to {OUTPUT_PKL}...")
    with open(OUTPUT_PKL, 'wb') as f:
        pickle.dump(pkl_sample, f)
    
    print("Done!")
    print(f"\nRun ShapeR with:")
    print(f"python ShapeR/infer_shape.py --input_pkl {OUTPUT_PKL} --config balance --is_local_path")

if __name__ == "__main__":
    main()

Loading MPS point cloud...
Loaded 19186 points
Loading trajectory...
Extracting frames and poses from VRS...


[ProgressLogger][INFO]: 2026-02-10 02:29:29: Opening data/b6591925-cd53-4ba9-b26c-eed4faffd8bd.vrs...
[MultiRecordFileReader][DEBUG]: Opened file 'data/b6591925-cd53-4ba9-b26c-eed4faffd8bd.vrs' and assigned to reader #0
[VrsDataProvider][INFO]: streamId 211-1/camera-et activated
[VrsDataProvider][INFO]: streamId 214-1/camera-rgb activated
[VrsDataProvider][INFO]: streamId 231-1/mic activated
[VrsDataProvider][INFO]: streamId 247-1/baro0 activated
[VrsDataProvider][INFO]: Utc stream found: 285-1
[VrsDataProvider][INFO]: streamId 1201-1/camera-slam-left activated
[VrsDataProvider][INFO]: streamId 1201-2/camera-slam-right activated
[VrsDataProvider][INFO]: streamId 1202-1/imu-right activated
[VrsDataProvider][INFO]: streamId 1202-2/imu-left activated
[VrsDataProvider][INFO]: streamId 1203-1/mag0 activated


Extracted 16 views
Creating masks (using dummy masks - replace with SAM2)...
Computing point visibility...
Computing point projections...
Building pickle file...
Saving to data/output.pkl...
Done!

Run ShapeR with:
python ShapeR/infer_shape.py --input_pkl data/output.pkl --config balance --is_local_path


In [14]:
!conda install -y -c conda-forge sparsehash


Channels:
 - conda-forge
Platform: linux-64
doneecting package metadata (repodata.json): - 
doneing environment: / 


==> WARNING: A newer version of conda exists. <==
    current version: 25.1.0
    latest version: 26.1.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - sparsehash


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2026.1.4   |       hbd8a1cb_0         143 KB  conda-forge
    certifi-2026.1.4           |     pyhd8ed1ab_0         147 KB  conda-forge
    openssl-3.6.1              |       h35e630c_1         3.0 MB  conda-forge
    sparsehash-2.0.4           |       hcb278e6_1          84 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.4 MB

The fo

In [ ]:
!pip install git+https://github.com/mit-han-lab/torchsparse.git


  Cloning https://github.com/mit-han-lab/torchsparse.git to /tmp/pip-req-build-rio_xhhl
  Running command git clone --filter=blob:none --quiet https://github.com/mit-han-lab/torchsparse.git /tmp/pip-req-build-rio_xhhl
  Resolved https://github.com/mit-han-lab/torchsparse.git to commit 385f5ce8718fcae93540511b7f5832f4e71fd835
  Preparing metadata (setup.py) ... one

In [13]:
# First, completely uninstall torchsparse
!pip uninstall torchsparse -y

# Clean up any build artifacts
!rm -rf ~/local/include/google ~/local/include/sparsehash
!rm -rf sparsehash-*

# Reinstall system dependencies
!cd sparsehash-sparsehash-2.0.4 && ./configure --prefix=$HOME/local && make install

# Set environment variables
import os
os.environ['CPLUS_INCLUDE_PATH'] = os.path.expanduser('~/local/include')
os.environ['C_INCLUDE_PATH'] = os.path.expanduser('~/local/include')

# Reinstall torchsparse
!pip install git+https://github.com/mit-han-lab/torchsparse.git

Found existing installation: torchsparse 2.1.0
Uninstalling torchsparse-2.1.0:
  Successfully uninstalled torchsparse-2.1.0
/bin/bash: line 1: cd: sparsehash-sparsehash-2.0.4: No such file or directory
  Cloning https://github.com/mit-han-lab/torchsparse.git to /tmp/pip-req-build-6i1xj49d
  Running command git clone --filter=blob:none --quiet https://github.com/mit-han-lab/torchsparse.git /tmp/pip-req-build-6i1xj49d
  Resolved https://github.com/mit-han-lab/torchsparse.git to commit 385f5ce8718fcae93540511b7f5832f4e71fd835
  Preparing metadata (setup.py) ... one
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [96 lines of output]
      torchsparse version: 2.1.0
      running bdist_wheel
      /opt/conda/lib/python3.12/site-packages/torch/utils/cpp_extension.py:497: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
   

In [ ]:
!pip install xformers

  Using cached torch-2.10.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (31 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached cuda_bindings-12.9.4-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (2.6 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cudnn_cu12-9.10.2.21-py3-none-manylinux_2_27_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_cublas_cu12-12.8.4.1-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cufft_cu12-11.3.3.83-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_curand_cu12-10.3.9.90-py3-none-manylinux_2_27_x8

In [4]:
!conda create -y -n ts310 python=3.10


Channels:
 - conda-forge
Platform: linux-64
doneecting package metadata (repodata.json): - 
doneing environment: | 


==> WARNING: A newer version of conda exists. <==
    current version: 25.1.0
    latest version: 26.1.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda/envs/ts310

  added / updated specs:
    - python=3.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |            2_gnu          23 KB  conda-forge
    bzip2-1.0.8                |       hda65f42_8         254 KB  conda-forge
    ca-certificates-2026.1.4   |       hbd8a1cb_0         143 KB  conda-forge
    icu-78.2                   |       h33c6efd_0        12.1 MB  conda-forge
    ld_impl_linux-64-2.45.1    |default_hbd61a6

In [5]:
!conda run -n ts310 pip install torch==2.1.* torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.3 GB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.3 GB 25.1 MB/s eta 0:01:33
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.3 GB 48.1 MB/s eta 0:00:49
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.3 GB 18.6 MB/s eta 0:02:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.3 GB 29.2 MB/s eta 0:01:19
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.3 GB 27.9 MB/s eta 0:01:23
     ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.3 GB 27.1 MB/s eta 0:01:25
     ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.3 GB 26.6 MB/s eta 0:01:27
     ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.3 GB 26.2 MB/s eta 0:01:28
     ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.3 GB 25.9 MB/s eta 0:01:29
     ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.3 GB 25.6 MB/s eta 0:01:29
     ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/2.3 GB 25.4 MB/s eta 0:01:30
 

In [6]:
!conda run -n ts310 conda install -y -c conda-forge sparsehash


Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda/envs/ts310

  added / updated specs:
    - sparsehash


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libstdcxx-ng-15.2.0        |      hdf11a46_17          27 KB  conda-forge
    sparsehash-2.0.4           |       hcb278e6_1          84 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         111 KB

The following NEW packages will be INSTALLED:

  libstdcxx-ng       conda-forge/linux-64::libstdcxx-ng-15.2.0-hdf11a46_17 
  sparsehash         conda-forge/linux-64::sparsehash-2.0.4-hcb278e6_1 



sparsehash-2.0.4     | 84 KB     |            |   0% 

libstdcxx-ng-15.2.0  | 27 KB     |            |   0% 
sparsehash-2.0.4     | 84 KB     | #8         |  19% 
sparsehash-2

In [9]:
!conda run -n ts310 pip install "numpy<2"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/18.2 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━ 14.7/18.2 MB 80.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 18.1/18.2 MB 78.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 18.1/18.2 MB 78.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 18.1/18.2 MB 78.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 18.1/18.2 MB 78.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 18.1/18.2 MB 78.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 18.1/18.2 MB 78.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 18.1/18.2 MB 78.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 18.1/18.2 MB 78.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 9.0 MB/s  0:00:02
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.

In [10]:
!conda run -n ts310 pip install --upgrade setuptools wheel


In [13]:
!conda run -n ts310 pip install --upgrade pip


In [15]:
!conda run -n ts310 pip --version


pip 26.0.1 from /opt/conda/envs/ts310/lib/python3.10/site-packages/pip (python 3.10)



In [17]:
!git clone https://github.com/mit-han-lab/torchsparse.git


Cloning into 'torchsparse'...
remote: Enumerating objects: 1277, done.
remote: Counting objects: 100% (318/318), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 1277 (delta 219), reused 194 (delta 194), pack-reused 959 (from 1)
Receiving objects: 100% (1277/1277), 1.47 MiB | 18.10 MiB/s, done.
Resolving deltas: 100% (623/623), done.
Updating files: 100% (179/179), done.


In [18]:
!conda run -n ts310 python torchsparse/setup.py install


Traceback (most recent call last):
  File "/home/jovyan/torchsparse/setup.py", line 7, in <module>
    from torch.utils.cpp_extension import (
  File "/opt/conda/envs/ts310/lib/python3.10/site-packages/torch/utils/cpp_extension.py", line 28, in <module>
    from pkg_resources import packaging  # type: ignore[attr-defined]
ModuleNotFoundError: No module named 'pkg_resources'

ERROR conda.cli.main_run:execute(125): `conda run python torchsparse/setup.py install` failed. (See above for error)


In [19]:
!conda install -n ts310 -y setuptools


Channels:
 - conda-forge
Platform: linux-64
doneecting package metadata (repodata.json): - 
doneing environment: \ 


==> WARNING: A newer version of conda exists. <==
    current version: 25.1.0
    latest version: 26.1.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [21]:
!conda run -n ts310 python - << 'EOF'
import pkg_resources
print("pkg_resources OK")
EOF


/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `EOF')


/tmp/ipykernel_216/2201025900.py:2: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


pkg_resources OK


NameError: name 'EOF' is not defined

In [23]:
import sys
print(sys.executable)


/opt/conda/bin/python


In [28]:
!conda init

no change     /opt/conda/condabin/conda
no change     /opt/conda/bin/conda
no change     /opt/conda/bin/conda-env
no change     /opt/conda/bin/activate
no change     /opt/conda/bin/deactivate
no change     /opt/conda/etc/profile.d/conda.sh
no change     /opt/conda/etc/fish/conf.d/conda.fish
no change     /opt/conda/shell/condabin/Conda.psm1
no change     /opt/conda/shell/condabin/conda-hook.ps1
no change     /opt/conda/lib/python3.12/site-packages/xontrib/conda.xsh
no change     /opt/conda/etc/profile.d/conda.csh
no change     /home/jovyan/.bashrc
No action taken.


In [33]:
import os
import subprocess
import sys

def run(cmd):
    print(f"Running: {cmd}")
    result = subprocess.run(cmd, shell=True, text=True, capture_output=True)
    print(result.stdout)
    if result.returncode != 0:
        print(result.stderr)
    return result.returncode

# 1️⃣ Ensure TorchSparse source is cloned
if not os.path.exists("torchsparse"):
    run("git clone https://github.com/mit-han-lab/torchsparse.git")

# 2️⃣ Enter the directory
os.chdir("torchsparse")

# 3️⃣ Verify directory contents
print("Directory contents:", os.listdir())

# 4️⃣ Install TorchSparse from source
run(f"{sys.executable} -m pip install --no-build-isolation .")

# 5️⃣ Test import
try:
    import torchsparse
    print("✅ TorchSparse installed successfully!")
except Exception as e:
    print("❌ TorchSparse import failed:", e)


Running: git clone https://github.com/mit-han-lab/torchsparse.git

Directory contents: ['.git', '.github', '.gitignore', '.pre-commit-config.yaml', 'LICENSE', 'README.md', 'cython_setup.py', 'docs', 'examples', 'install.py', 'requirements.txt', 'setup.cfg', 'setup.py', 'tests', 'torchsparse']
Running: /opt/conda/bin/python -m pip install --no-build-isolation .
Processing ./.
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [59 lines of output]
      torchsparse version: 2.1.0
      running dist_info
      creating /tmp/pip-modern-metadata-lph7yvae/torchsparse.egg-info
      writing /tmp/pip-modern-metadata-lph7yvae/torchsparse.egg-info/PKG-INFO
      writing dependency_links to /tmp/pip-modern-metadata-lph7yvae/torchsparse.egg-info/dependency_links.txt
      writing requirements to /tmp/pip-m

In [34]:
import sys
import subprocess
import os

# Helper function to run shell commands
def run(cmd):
    print(f"\n>> {cmd}")
    result = subprocess.run(cmd, shell=True, text=True, capture_output=True)
    print(result.stdout)
    if result.returncode != 0:
        print(result.stderr)
    return result.returncode

# 1️⃣ Ensure pkg_resources (part of setuptools) is installed
run(f"{sys.executable} -m pip install --upgrade setuptools wheel pbr")

# 2️⃣ Clone TorchSparse if missing
if not os.path.exists("torchsparse"):
    run("git clone https://github.com/mit-han-lab/torchsparse.git")

# 3️⃣ Go to the TorchSparse folder (it contains setup.py)
os.chdir("torchsparse")

# 4️⃣ List files to confirm setup.py exists
print("Files in TorchSparse folder:", os.listdir())

# 5️⃣ Install TorchSparse without build isolation
run(f"{sys.executable} -m pip install --no-build-isolation .")

# 6️⃣ Test import
try:
    import torchsparse
    print("✅ TorchSparse installed successfully!")
except Exception as e:
    print("❌ TorchSparse import failed:", e)



>> /opt/conda/bin/python -m pip install --upgrade setuptools wheel pbr
  Attempting uninstall: pbr
    Found existing installation: pbr 6.1.1
    Uninstalling pbr-6.1.1:
      Successfully uninstalled pbr-6.1.1

Files in TorchSparse folder: ['__init__.py', 'backbones', 'backend', 'backends.py', 'nn', 'operators.py', 'tensor.py', 'utils', 'version.py', '__pycache__']

>> /opt/conda/bin/python -m pip install --no-build-isolation .

ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.

✅ TorchSparse installed successfully!


In [35]:
import os
os.chdir("..")  # move to the root folder of the repo


In [36]:
os.listdir()
# You should see: setup.py  LICENSE  README.md  torchsparse/


['.git',
 '.github',
 '.gitignore',
 '.pre-commit-config.yaml',
 'LICENSE',
 'README.md',
 'cython_setup.py',
 'docs',
 'examples',
 'install.py',
 'requirements.txt',
 'setup.cfg',
 'setup.py',
 'tests',
 'torchsparse']

In [37]:
!{sys.executable} -m pip install --no-build-isolation .


Processing ./.
  Preparing metadata (pyproject.toml) ... one
  Using cached backports.cached_property-1.0.2-py3-none-any.whl.metadata (3.8 kB)
Using cached backports.cached_property-1.0.2-py3-none-any.whl (6.1 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for torchsparse (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [184 lines of output]
      torchsparse version: 2.1.0
      running bdist_wheel
      running build
      running build_py
      creating build/lib.linux-x86_64-cpython-312/torchsparse
      copying torchsparse/__init__.py -> build/lib.linux-x86_64-cpython-312/torchsparse
      copying torchsparse/backends.py -> build/lib.linux-x86_64-cpython-312/torchsparse
      copying torchsparse/operators.py -> build/lib.linux-x86_64-cpython-312/torchsparse
      copying torchsparse/tensor.py -> build/lib.linux-x86_64-cpython-312/torchsparse
      copying torchsparse/version.py -> build/lib.linux-x86_64-cpython-312/torchsparse
      creating build

In [ ]:
# Force classic solver
import os
os.environ['CONDA_SUBDIR'] = 'linux-64'
os.environ['CONDA_SOLVER'] = 'classic'
os.environ['CONDA_NO_PLUGINS'] = 'true'

!conda install -c conda-forge google-sparsehash --solver=classic -y


In [3]:
!python ShapeR/infer_shape.py --input_pkl data/output.pkl --config balance --is_local_path

Traceback (most recent call last):
  File "/home/jovyan/ShapeR/infer_shape.py", line 28, in <module>
    from dataset.shaper_dataset import InferenceDataset
  File "/home/jovyan/ShapeR/dataset/shaper_dataset.py", line 12, in <module>
    import torchsparse
ModuleNotFoundError: No module named 'torchsparse'
